In [5]:
import ccxt
from datetime import datetime
import plotly.graph_objects as go# collect the candlestick data from Binance
binance = ccxt.binance()
trading_pair = 'LTC/BTC'
candles = binance.fetch_ohlcv(trading_pair, '1h')
dates = []
open_data = []
high_data = []
low_data = []
close_data = []# format the data to match the charting library
for candle in candles:
    dates.append(datetime.fromtimestamp(candle[0] / 1000.0).strftime('%Y-%m-%d %H:%M:%S.%f'))
    open_data.append(candle[1])
    high_data.append(candle[2])
    low_data.append(candle[3])
    close_data.append(candle[4])# plot the candlesticks

layout=go.Layout(title= 'iShares MSCI Emerging Markets ETF (EEM)',
                 xaxis=dict(title='Date', 
                            rangeslider=dict(visible=False)),
                    yaxis=dict(title='Price', 
                               showgrid=True,
                               color='black',
                                gridwidth=1, 
                                gridcolor='LightPink'),
                    plot_bgcolor='black',
                    paper_bgcolor='lightgray',
                 
                )
fig = go.Figure(data=[go.Candlestick(x=dates,
                       open=open_data, high=high_data,
                       low=low_data, close=close_data)], layout=layout)
fig.show()

In [9]:
import plotly.graph_objects as go
import pandas as pd


raw_candles = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
raw_candles = raw_candles[-100:].copy() # more candles, havier the plot
raw_candles.head(1)

,Date,AAPL.Open,AAPL.High,AAPL.Low,AAPL.Close,AAPL.Volume,AAPL.Adjusted,dn,mavg,up,direction
406,2016-09-26,111.639999,113.389999,111.550003,112.879997,29869400,111.818608,102.184292,109.680667,117.177042,Increasing


In [10]:
new_names = {
    'AAPL.Open': 'open',
    'AAPL.High': 'high',
    'AAPL.Low': 'low',
    'AAPL.Close': 'close',
}
raw_candles.rename(columns=new_names, inplace=True)
raw_candles.set_index('Date', inplace=True)


In [11]:
# My Art! we define some variables in order to make the code undertandable

play_button = {
    "args": [
        None, 
        {
            "frame": {"duration": 300, "redraw": True},
            "fromcurrent": True, 
            "transition": {"duration": 100,"easing": "quadratic-in-out"}
        }
    ],
    "label": "Play",
    "method": "animate"
}

pause_button = {
    "args": [
        [None], 
        {
            "frame": {"duration": 0, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 0}
        }
    ],
    "label": "Pause",
    "method": "animate"
}

sliders_steps = [
    {"args": [
        [0, i], # 0, in order to reset the image, i in order to plot frame i
        {"frame": {"duration": 300, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
    "label": i,
    "method": "animate"}
    for i in range(len(raw_candles))      
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "candle:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": sliders_steps,
}

initial_plot = go.Candlestick(
    x=raw_candles.index, 
    open=raw_candles.open, 
    high=raw_candles.high, 
    low=raw_candles.low, 
    close=raw_candles.close
)

first_i_candles = lambda i: go.Candlestick(
    x=raw_candles.index, 
    open=raw_candles.open[:i], 
    high=raw_candles.high[:i], 
    low=raw_candles.low[:i], 
    close=raw_candles.close[:i]
)

fig = go.Figure(
    data=[initial_plot],
    layout=go.Layout(
        xaxis=dict(title='Date', rangeslider=dict(visible=False)),
        title="Candles over time",
        updatemenus= [dict(type="buttons", buttons=[play_button, pause_button])],
        sliders = [sliders_dict]
    ),
    frames=[
        go.Frame(data=[first_i_candles(i)], name=f"{i}") # name, I imagine, is used to bind to frame i :) 
        for i in range(len(raw_candles))
    ]
)

fig.show()


In [12]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = go.Figure(data=[go.Candlestick(x=df['Date'][400:],
                open=df['AAPL.Open'][400:], high=df['AAPL.High'][400:],
                low=df['AAPL.Low'][400:], close=df['AAPL.Close'][400:])
                     ])

fig.add_trace(go.Scatter(x=df['Date'][400:],
                         y=[df['AAPL.Low'][400:].min()]*len(df['Date'][400:]),
                         mode="lines",
                         line=dict(color='red', width=2)))

fig.add_trace(go.Scatter(x=df['Date'][400:],
                         y=[df['AAPL.High'][400:].min()]*len(df['Date'][400:]),
                         mode="lines",
                         line=dict(color='blue', width=2)))
last = 0
for i in range(len(df['Date'][400:])):
    temp = df.loc[400+i:400+i,'AAPL.Low'].values[0]
    if temp >= last:
        fig.add_annotation(x=df.loc[400+i,'Date'], y=temp+3, text="▲", showarrow=False, font=dict(size=16, color='LightSeaGreen'))
        last = temp
    else:
        fig.add_annotation(x=df.loc[400+i,'Date'], y=temp-3, text="▲", showarrow=False, font=dict(size=16, color='red'))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()